We will now use the TGNN architecture employed in other files as a basis to build a model that is able to perform Emotional Recognition Classification. This task involves classifying the emotional state of speaker utterances in multi-party conversation. 

Lets first define the pytorch geomrtic dataset. 

The dataset object taked in a list of graphs where each graph, 'G', is composed of a sub-graphs 'G_i' where each subgraph represents an utterance.  

Each subggraph G_i has an entry for Structure, 'X', and one for its emotion label, 'Y'. 

X contains node entries where each node has a corresponding embedding and list of edges. 

Consider entry 'X' in graph 'G_0':

A dictionary entry for a speaker node will have a full feature embedding and edges pointing to all other nodes in the time stamp as well as an edge pointing to its future state. 
- Ex. 'D_0': {'embedding': [-0.46728479862213135, -0.20498991012573242, -0.43848446011543274, ....] ,'edges': ['A_0', 'B_0', 'C_0']}

A dictionary entry for a silent node will have a null feature embedding and one edge pointing to its future state. 
- Ex. 'A_0': {'embedding': [NULL] ,'edges': []}


In [1]:
import torch
from torch_geometric.data import Data, Dataset

class ERC_Dataset(Dataset):
    def __init__(self, data_list):
        super(ERC_Dataset, self).__init__()
        self.data_list = []

        for entry in data_list:
            graph_data = entry['G']  
            node_names = list(graph_data.keys())  


            node_features = []
            edge_index = []

            # Iterate over each node in the graph to collect embeddings and edge info
            for node in node_names:
                # Get the node embedding and edges
                node_embedding = graph_data[node]['embedding']
                edges = graph_data[node]['edges']

                # Append node embedding
                node_features.append(node_embedding)

                # Convert node names into indices and append to edge index
                src_idx = node_names.index(node)
                for edge_target in edges:
                    if edge_target in node_names:
                        dst_idx = node_names.index(edge_target)
                        edge_index.append([src_idx, dst_idx])

                # Convert node features and edge indices to tensors
                x = torch.tensor(node_features, dtype=torch.float)
                edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

                # Store the binary label as the y label
                y = torch.tensor([entry['cohesion_score']], dtype=torch.float)

                # Create Geometric Data object with node names as an additional attribute
                data = Data(x=x, edge_index=edge_index, y=y)
                data.node_names = node_names  # Store node names in the Data object

                self.data_list.append(data)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]



Forward passing should take on the following update steps: 


For G_i in dataset[i]:
    1. update node memories for all listeners replacing null embeddings for memory embedidngs
    2. apply message passing in G_i

step 1 should call a function with dataset[i] and G_i as a paramete. The function will make put together a sequence of memory states for each listener node and predictthe currnet one which will be used to replace the null embeddings. 


In [ ]:
For